# Homework 1 - Linear Regression

## Dataset
The dataset you will be using is the "Bike Sharing". 

There are two data files: "BikeSharing_training.csv" and "BikeSharing_Xtest.csv"<br/>
Both files have the following fields, except cnt which is not available in "BikeSharing_Xtest.csv"

Features:
- season : season (1:winter, 2:spring, 3:summer, 4:fall)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
+ weathersit :
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)

Target:
- cnt: count of total rental bikes


Training dataset, "BikeSharing_training.csv", contains 300 rows and 12 columns. This is the training set containing both of the features and the target.<br/>
Test dataset, "BikeSharing_Xtest.csv", contains 200 rows and 11 columns. This is the test set which only contains the features.<br/>

Your goal is to predict the number of total rental bikes (cnt) based on the features.

In [2]:
import csv
import numpy as np
import pandas as pd

Load the training data and View the first 5 lines

In [3]:
# Load the data
Tr = pd.read_csv("C:/Users/jason/Desktop/UCLA/401/BikeSharing_training.csv")
Te = pd.read_csv("C:/Users/jason/Desktop/UCLA/401/BikeSharing_Xtest.csv")
# Show the first 5 lines
Tr.head()

,season,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,12,16,0,5,1,1,0.42,0.4242,0.47,0.1940,283
1,4,10,9,0,0,0,2,0.50,0.4848,0.55,0.4179,330
2,3,9,1,0,0,0,2,0.62,0.5606,0.88,0.0000,88
3,3,6,22,0,3,1,1,0.70,0.6364,0.42,0.1940,183
4,3,7,12,0,1,1,2,0.80,0.7424,0.52,0.1642,314


## Data Exploration
We can plot a histogram of the dataframe for the features: "temp", "atemp","hum","windspeed" to understand the distributions of the continuous values.<br/>


In [4]:
### WRITE CODE TO OBTAIN AND DISPLAY HISTOGRAMS ###
temp_hist = Tr.hist(column = "temp")
atemp_hist = Tr.hist(column = "atemp")
hum_hist = Tr.hist(column = "hum")
windspeed_hist = Tr.hist(column = "windspeed")

##### Q1. What can you infer from the histograms? <br/>
Ans- Both hum and atemp are pretty noramlly distributed. temp has a significant number of occurance of 0.6-0.7. Windspeed is skewed very much to the right, with majority of cases happending in the 0 - 0.3 range.

Compute the correlation matrix to get an understanding of the correlation between cnt and the other features.<br/>



In [5]:
Tr.corr()

,season,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
season,1.000000,0.795990,-0.046306,-0.057180,-0.049248,0.087355,0.097812,0.403317,0.406946,0.155086,-0.110663,0.225385
mnth,0.795990,1.000000,0.000608,-0.012655,0.001790,0.045749,0.098688,0.270690,0.265519,0.116613,-0.005633,0.144026
hr,-0.046306,0.000608,1.000000,0.104970,0.029337,-0.083801,-0.117088,0.049154,0.042834,-0.375612,0.162829,0.387503
holiday,-0.057180,-0.012655,0.104970,1.000000,-0.065359,-0.216498,-0.059273,0.023122,0.020936,-0.064204,-0.043506,0.018659
weekday,-0.049248,0.001790,0.029337,-0.065359,1.000000,0.096844,-0.088959,-0.057797,-0.071621,-0.098027,0.073404,0.067401
workingday,0.087355,0.045749,-0.083801,-0.216498,0.096844,1.000000,0.131374,-0.007330,-0.001266,0.168470,-0.000534,-0.072353
weathersit,0.097812,0.098688,-0.117088,-0.059273,-0.088959,0.131374,1.000000,-0.014367,-0.018496,0.403589,-0.047412,-0.167125
temp,0.403317,0.270690,0.049154,0.023122,-0.057797,-0.007330,-0.014367,1.000000,0.992783,-0.037122,-0.096506,0.435036
atemp,0.406946,0.265519,0.042834,0.020936,-0.071621,-0.001266,-0.018496,0.992783,1.000000,-0.017675,-0.126852,0.430083
hum,0.155086,0.116613,-0.375612,-0.064204,-0.098027,0.168470,0.403589,-0.037122,-0.017675,1.000000,-0.228720,-0.321508


##### Answer the following questions:<br/>

##### Q2. Why is the diagonal made up of 1's in the correlation matrix?<br/>
Ans - They are the correlation between an attribute with itself, which has to be 100% correlated.

##### Q3. Why is the matrix symmetric along diagonal?<br/>
Ans - The matrix is A compare to B on the lwft side and B compare to A on the right side of the diagonal. The correlation of A to B and B to A is the same.

##### Q4. Looking at the correlation matrix, if you have to choose one predictor for a simple linear regression model with cnt as the outcome, which one would you choose and why? <br/>
Ans - "temp", it has the highest absolute number of correlation, which indicates that it has the biggest impact on "cnt" holding all other attributes equal.


### Standardization of features

Feature standardization makes the values of each feature in the data have zero-mean and unit-variance. This method is widely used for normalization in many machine learning algorithms. The general method of calculation is to determine the distribution mean and standard deviation for each feature. Next we subtract the mean from each feature. Then we divide the values of each feature by its standard deviation.

$x'$ = ($x$ - $\bar{x}$)/$\sigma$ 

where $x$ is the original feature vector,
$\bar{x}$ is the mean of the feature vector and
$\sigma$ is its standard deviation.

This is also called Z-score Normalization. 

Perform Z-score Normalization on "temp", "atemp","hum","windspeed"


In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Z_temp = scaler.fit_transform(np.array(Tr['temp']).reshape(-1, 1))
Z_atemp = scaler.fit_transform(np.array(Tr['atemp']).reshape(-1, 1))
Z_hum = scaler.fit_transform(np.array(Tr['hum']).reshape(-1, 1))
Z_windspeed = scaler.fit_transform(np.array(Tr['windspeed']).reshape(-1, 1))

##### Q5. What are the advantages and disadvantages of using Z-score Normalization?<br/>
Ans- Advantages of Z-score normalization are that it allows us to see the probability of a particular value occuring in the sample and that it allows us to compare samples of the same attribute. One disadvantage is that it assumes normal distribution even when the sample is not normally distributed.

##### Q6. In this dataset, do you need to use the Z-score Normalization? Explain.<br/>
Ans- No, because we only have one sample for an attribute and many attributes are very skewed.

### One-Hot Encoding

"temp", "atemp","hum" and "windspeed" are continuous values whereas the others contain discrete values. E.g. "mnth" can only take on the integers from 1 to 12. We need to perform one-hot encoding on discrete values for it to be processed in the model. One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction.

Perform one-hot encoding on all the categorical features and print the shape of your encoded array

In [7]:
from sklearn.preprocessing import OneHotEncoder
categories = Tr.columns[0:7]
label = Tr[categories]
enc = OneHotEncoder(categories='auto')
encoded = enc.fit_transform(label).toarray()

new_columns = list()
for col, values in zip(label.columns, enc.categories_):
    new_columns.extend([col + '_' + str(value) for value in values])

coded_Tr = Tr.drop(columns = categories)
coded_Tr = pd.concat([pd.DataFrame(encoded, columns = new_columns), coded_Tr],axis = 'columns')

coded_Tr.head(5)
print(encoded.shape)

(300, 54)


##### Q7. What are the advantages and disadvantages of using One-hot encoding?<br/>
Ans- One advantage is that the result is binary, enabling differentiation between categorical datas. The disadvantage is that the number of output can be too large.

## Multiple Linear Regression

In the big data era, it is highly unlikely that we are interested in the effect of a single variable on another. To simultaneously account for the effects of multiple variables, we use multiple regression (which accounts for the covariances between predictors).

While the algorithmic solution to multiple regression exists, it is easier to conceptualize in terms of linear algebra. The optimal $\hat{\beta}$ vector that minimizes the residual sum of squares is:

$\hat{\beta} = (X^TX)^{-1}X^Ty $


Perform multiple linear regression on the training dataset, where the outcome is "cnt".

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
# Bulding and fitting the Linear Regression model
dep = coded_Tr.columns[:-1]
x = coded_Tr[dep]
y = coded_Tr['cnt']
mlr = LinearRegression()
mlr.fit(x,y)

# Evaluating the Linear Regression model by computing MSE on training set
from sklearn.metrics import mean_squared_error
length = len(coded_Tr)
predict = []
true = coded_Tr['cnt'].tolist()
for x in range(length):
    predict.append(mlr.intercept_ + sum(coded_Tr[dep].loc[x].tolist() * mlr.coef_))
msr = mean_squared_error(predict, true)

###### Q8. Print the value of bhat

In [10]:
print(mlr.intercept_)
print(mlr.coef_)
print(msr)

-272377202290970.0
[ 3.48990062e+12  3.48990062e+12  3.48990062e+12  3.48990062e+12
  1.99610274e+14  1.99610274e+14  1.99610274e+14  1.99610274e+14
  1.99610274e+14  1.99610274e+14  1.99610274e+14  1.99610274e+14
  1.99610274e+14  1.99610274e+14  1.99610274e+14  1.99610274e+14
  9.98255180e+13  9.98255180e+13  9.98255180e+13  9.98255180e+13
  9.98255180e+13  9.98255180e+13  9.98255180e+13  9.98255180e+13
  9.98255180e+13  9.98255180e+13  9.98255180e+13  9.98255180e+13
  9.98255180e+13  9.98255180e+13  9.98255180e+13  9.98255180e+13
  9.98255180e+13  9.98255180e+13  9.98255180e+13  9.98255180e+13
  9.98255180e+13  9.98255180e+13  9.98255180e+13  9.98255180e+13
  2.35193390e+15 -2.10203358e+15 -3.76505954e+15  6.88907930e+14
  6.88907930e+14  6.88907930e+14  6.88907930e+14  6.88907930e+14
 -3.76505954e+15  2.46128273e+15 -1.99268474e+15 -1.07870557e+15
 -1.07870557e+15 -1.07870557e+15  8.19843750e+02 -5.16750000e+02
 -4.98730469e+01 -1.05077148e+02]
11287.805126953124


###### Q9. Is there a problem of multicolinearity? Explain what you can do<br/>
Ans- msr gets so big I do not know why.

### Goodness of fit

A model can always make predictions. But it is important to determine how good the model is.
How do we know that our model captures the data well? When evaluating model fit, a good metric is $R^2$, which corresponds to the amount of variance explained by the model. The formula for $R^2$ is the following:

$R^2$ = $1 - \dfrac{RSS}{TSS}$<br/>
where:<br/>
$RSS = \Sigma(y - \hat{y})^2$<br/>
$TSS = \Sigma(y - \bar{y})^2$<br/>

$R^2$ is also one metric for comparing models against each other. It is intuitive to say that the model that explains more variation in the data is a better fit than one that explains less variation. 

Fill in the code for calculation of R2 score 

In [11]:
from sklearn.metrics import r2_score

#### R2 score for model with "temp" as predictor and "cnt" as outcome

In [12]:
x_temp = np.array(coded_Tr['temp']).reshape(-1, 1)
lr_temp = LinearRegression().fit(x_temp, y)

predict_with_temp = []
for x in range(length):
    predict_with_temp.append(lr_temp.intercept_ + coded_Tr['temp'].loc[x] * lr_temp.coef_)
# Print R2 score
r2_temp = r2_score(true, predict_with_temp)
print(r2_temp)

0.18925663564676132


#### R2 score for model with "temp", "hum" as predictors and "cnt" as outcome

In [13]:
x_temp_hum = coded_Tr[['temp', 'hum']]
lr_temp_hum = LinearRegression().fit(x_temp_hum, y)

pred_temp_hum = []
for x in range(length):
    pred_temp_hum.append(lr_temp_hum.intercept_ + sum(x_temp_hum.loc[x].tolist() * lr_temp_hum.coef_))
# Print R2 score
r2_temp_hum = r2_score(true, pred_temp_hum)
print(r2_temp_hum)

0.2826295103178843


#### R2 score for model with  "temp", "atemp", "hum" as predictors and "cnt" as outcome

In [14]:
x_temp_hum_atemp = coded_Tr[['temp', 'hum','atemp']]
lr_temp_hum_atemp = LinearRegression().fit(x_temp_hum_atemp, y)

pred_temp_hum_atemp = []
for x in range(length):
    pred_temp_hum_atemp.append(lr_temp_hum_atemp.intercept_ + sum(x_temp_hum_atemp.loc[x].tolist() * lr_temp_hum_atemp.coef_))
# Print R2 score
r2_temp_hum_atemp = r2_score(true, pred_temp_hum_atemp)
print(r2_temp_hum_atemp)

0.28380026219213783


You can see $R^2$ is always going up as we keep adding features.

This is one drawback of only using $R^2$ to evaluate your model. Adding predictors will always improve the predictive ability of your model, though it may not be meaningful.

That is to say, we are not necessarily interested in making a perfect prediciton of our data. If we were, we would always use all of the predictors available. Rather, we are interested in the most predictive features, in the hopes that we can create a simpler model that performs nearly as well.

This is why we consider another metric, Adjusted R2.
The adjusted R-squared increases only if the new term improves the model more than would be expected by chance.
Using Adjusted R2 score, please give the most relevant combination of predictors in determining the profit of the startup.


$AdjustedR^2$ = $1 - \dfrac{(1-R^2)(n-1)}{(n-k-1)}$<br/>
where:<br/>
n = number of samples<br/>
k = number of features

Fill in the code for calculation of adjusted R2 score

#### Adjusted R2 score for model with "temp" as predictor and "cnt" as outcome

In [15]:
ar2_temp = 1 - (1 - r2_temp) * (length - 1) / (length - 1 - 1)

# Print Adjusted R2 score
print(ar2_temp)

0.18653602033013972


#### Adjusted R2 score for model with "temp", "hum" as predictors and "cnt" as outcome

In [16]:
ar2_temp_hum = 1 - (1 - r2_temp_hum) * (length - 1) / (length - 2 - 1)

# Print Adjusted R2 score
print(ar2_temp_hum)

0.2777987326095872


#### Adjusted R2 score for model with  "temp", "atemp", "hum" as predictors and "cnt" as outcome

In [17]:
ar2_temp_hum_atemp = 1 - (1 - r2_temp_hum_atemp) * (length - 1) / (length - 3 - 1)

# Print Adjusted R2 score
print(ar2_temp_hum_atemp)

0.27654148106570675


### K-fold Cross-Validation

However, adjusted R2 is not enough to help us ahieve the best model, a more robust method is k-fold cross-validation.

* Randomly split dataset into K equal-sized subsets, or folds
* Treat each fold as validation set (train on all but K'th fold and test on K'th fold only)

* The overall error is then the mean error over all K models.
* Most common are 5- or 10-fold cross-validation

Please implement a 5-fold cross-validation by yourselves to find the best model in terms of Mean Square Error(MSE)

**Do not use sklearn.model_selection.cross_val_score or other built-in cross-validaiton functions**

In [18]:
# Design a function to implement 5-fold cross-validation. 
# The input: training features X, training target y.
# The output: the average of MSE over the 5 folds.

def cross_val_mse(X, y):
    # Write your code here
    msr = []
    for i in range(5):
        x_test = X.iloc[int(i * 300 / 5) : int((i + 1) * 300 / 5),:]
        x_train = X.drop(X.index[int(i * 300 / 5) : int((i + 1) * 300 / 5)])
        
        y_test = y.iloc[int(i * 300 / 5) : int((i + 1) * 300 / 5),:]
        y_train = y.drop(y.index[int(i * 300 / 5) : int((i + 1) * 300 / 5)])
        
        reg = LinearRegression().fit(x_train, y_train)
        predict = reg.predict(x_test)
        
        msr.append(mean_squared_error(y_test, predict))
    
    return sum(msr) / len(msr)

# By using your above function, find the best combination of features, which has the lowest averaged MSE

from itertools import combinations

cols = [i for i in Tr.columns if i != 'cnt']

all_comb = []
for i in range(len(cols)):
    all_comb += list(combinations(cols, i))

all_msr = []
for n, i in enumerate(range(1, len(all_comb)), 0):
    all_msr.append([cross_val_mse(Tr[list(all_comb[i])], Tr[['cnt']]), n])

best_comb = all_comb[min(all_msr)[1]]

In [19]:
# Print the best features 
print(best_comb)

('season', 'mnth', 'hr', 'weekday', 'weathersit', 'temp', 'windspeed')


In [26]:
Te.head()

,season,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,4,11,11,1,4,0,1,0.42,0.4242,0.41,0.0896
1,1,12,13,0,5,1,2,0.30,0.3030,0.49,0.1343
2,4,11,18,0,3,1,1,0.44,0.4394,0.67,0.1045
3,4,10,16,0,4,1,1,0.70,0.6515,0.54,0.2836
4,4,12,17,0,0,0,1,0.28,0.3182,0.30,0.0000


In [25]:
Tr.head()

,season,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,12,16,0,5,1,1,0.42,0.4242,0.47,0.1940,283
1,4,10,9,0,0,0,2,0.50,0.4848,0.55,0.4179,330
2,3,9,1,0,0,0,2,0.62,0.5606,0.88,0.0000,88
3,3,6,22,0,3,1,1,0.70,0.6364,0.42,0.1940,183
4,3,7,12,0,1,1,2,0.80,0.7424,0.52,0.1642,314


### Test your model
Now, apply your best model to predict the target values from the test feature set "BikeSharing_Xtest.csv". We will grade this part based on your prediction error.

Hint: Please be careful on standardization and one-hot encoding (if you use), the test set should be consistent with the training set in terms of any transformation.

Hint2: You may want to modify the previous steps to make the transformation of the test set consistent with the training set.

In [21]:
length = len(Te)
predict_Te = []
x = Tr[list(best_comb)]
y = Tr['cnt']
final_mlr = LinearRegression().fit(x,y)

for x in range(length):
    predict_Te.append(final_mlr.intercept_ + sum(Te[list(best_comb)].loc[x].tolist() * final_mlr.coef_))

# Output your prediction on test set as y_pred. It should be a 200 x 1 vector.
y_pred = predict_Te

In [22]:
y_pred[:5]

[204.70272946889457,
 57.35363738817861,
 275.13968557827314,
 366.3643361375511,
 178.16084574278267]

In [23]:
#end